In [20]:
import os
import re
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import torch
import torchvision.transforms.functional as TF
import tifffile
import yaml
from einops import rearrange


In [21]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
%autoreload 2

In [23]:
from mushroom.data.multiplex import extract_ome_tiff, write_basic_ome_tiff, get_ome_tiff_channels, make_pseudo
from mushroom.data.visium import adata_from_visium
from mushroom.visualization.utils import display_sections
import mushroom.utils as utils

In [24]:
run_dir = '/data/estorrs/mushroom/data/projects/subclone'
mushroom_dir = os.path.join(run_dir, 'mushroom')
Path(mushroom_dir).mkdir(parents=True, exist_ok=True)

In [25]:
metadata = yaml.safe_load(open(os.path.join(run_dir, 'registered', 'metadata.yaml')))
metadata

{'HT112C1': {'data': {'visium': {'HT112C1-U1': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registered/HT112C1/s0_visium_processed.h5ad',
    'HT112C1-U2': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registered/HT112C1/s1_visium_processed.h5ad'}},
  'ids': ['s0', 's1'],
  'order': ['HT112C1-U1', 'HT112C1-U2'],
  'registered_pixels_per_micron': 2.480402478562179,
  'scale': 0.1},
 'HT206B1': {'data': {'visium': {'HT206B1-U2': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registered/HT206B1/s0_visium_processed.h5ad',
    'HT206B1-U3': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registered/HT206B1/s1_visium_processed.h5ad',
    'HT206B1-U4': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registered/HT206B1/s2_visium_processed.h5ad',
    'HT206B1-U5': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/subclone/registered/HT206B1/s3_visium_processed.h5ad'}},
  'ids': ['s0', 's1', 's2', 's

In [14]:
configs = {}
for case, data in metadata.items():
    config = [
        {
            'id': sid,
            'position': 0,
            'data': []
        }
        for sid in data['ids']
    ]
    
    for dtype, mapping in data['data'].items():
        for sample_id, filepath in mapping.items():
            idx = data['order'].index(sample_id)
            
            # convert from katmai to glacier
            filepath = filepath.replace('/diskmnt/Projects/Users/estorrs', '/data/estorrs')
            
            config[idx]['data'].append(
                {
                    'dtype': dtype,
                    'filepath': filepath,
                }
            )
    configs[case] = config
configs
    

{'HT112C1': [{'id': 's0',
   'position': 0,
   'data': [{'dtype': 'visium',
     'filepath': '/data/estorrs/mushroom/data/projects/subclone/registered/HT112C1/s0_visium_processed.h5ad'}]},
  {'id': 's1',
   'position': 0,
   'data': [{'dtype': 'visium',
     'filepath': '/data/estorrs/mushroom/data/projects/subclone/registered/HT112C1/s1_visium_processed.h5ad'}]}],
 'HT206B1': [{'id': 's0',
   'position': 0,
   'data': [{'dtype': 'visium',
     'filepath': '/data/estorrs/mushroom/data/projects/subclone/registered/HT206B1/s0_visium_processed.h5ad'}]},
  {'id': 's1',
   'position': 0,
   'data': [{'dtype': 'visium',
     'filepath': '/data/estorrs/mushroom/data/projects/subclone/registered/HT206B1/s1_visium_processed.h5ad'}]},
  {'id': 's2',
   'position': 0,
   'data': [{'dtype': 'visium',
     'filepath': '/data/estorrs/mushroom/data/projects/subclone/registered/HT206B1/s2_visium_processed.h5ad'}]},
  {'id': 's3',
   'position': 0,
   'data': [{'dtype': 'visium',
     'filepath': '/dat

In [16]:
# filling out position seperately and reading back in
# yaml.safe_dump(configs, open(os.path.join(mushroom_dir, 'configs_template.yaml'), 'w'))

In [18]:
configs = yaml.safe_load(open(os.path.join(mushroom_dir, 'configs_template.yaml')))
configs

{'HT112C1': [{'data': [{'dtype': 'visium',
     'filepath': '/data/estorrs/mushroom/data/projects/subclone/registered/HT112C1/s0_visium_processed.h5ad'}],
   'id': 's0',
   'position': 0},
  {'data': [{'dtype': 'visium',
     'filepath': '/data/estorrs/mushroom/data/projects/subclone/registered/HT112C1/s1_visium_processed.h5ad'}],
   'id': 's1',
   'position': 60}],
 'HT206B1': [{'data': [{'dtype': 'visium',
     'filepath': '/data/estorrs/mushroom/data/projects/subclone/registered/HT206B1/s0_visium_processed.h5ad'}],
   'id': 's0',
   'position': 0},
  {'data': [{'dtype': 'visium',
     'filepath': '/data/estorrs/mushroom/data/projects/subclone/registered/HT206B1/s1_visium_processed.h5ad'}],
   'id': 's1',
   'position': 40},
  {'data': [{'dtype': 'visium',
     'filepath': '/data/estorrs/mushroom/data/projects/subclone/registered/HT206B1/s2_visium_processed.h5ad'}],
   'id': 's2',
   'position': 80},
  {'data': [{'dtype': 'visium',
     'filepath': '/data/estorrs/mushroom/data/projec

In [19]:
for case, config in configs.items():
    directory = os.path.join(mushroom_dir, case)
    Path(directory).mkdir(parents=True, exist_ok=True)
    
    yaml.safe_dump(config, open(os.path.join(directory, 'config.yaml'), 'w'))